In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib

#system
from pathlib import Path
import time

#ai
import torch
from torch import nn
import torchvision
from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import src
importlib.reload(src)

import src.utils.metrics as metrics
import src.model.train_recurrent as train_recurrent
import src.model.ae as ae
import src.model.ae_recurrente as ae_recurrente
import src.utils.gau as gaussian
import src.probing.linear_probing as lp
import src.utils.plots as plots


In [68]:
stamps_dataset = pd.read_pickle('data/5stamps_dataset.pkl')
#dataset_1_21 = pd.read_pickle('data/stamp_dataset_21_new.pkl')

In [69]:
def rename_labels(dataset, old_value, new_value):
    for key in dataset.keys():
        if old_value in dataset[key]:
            dataset[key][new_value] = dataset[key].pop(old_value)

rename_labels(stamps_dataset, 'labels', 'class')
rename_labels(stamps_dataset, 'science', 'images')

In [ ]:
train_template = torch.tensor(stamps_dataset['Train']['template'], dtype=torch.float32)
validation_template = torch.tensor(stamps_dataset['Validation']['template'], dtype=torch.float32)
test_template = torch.tensor(stamps_dataset['Test']['template'], dtype=torch.float32)

train_difference = torch.tensor(stamps_dataset['Train']['difference'], dtype=torch.float32)
validation_difference = torch.tensor(stamps_dataset['Validation']['difference'], dtype=torch.float32)
test_difference = torch.tensor(stamps_dataset['Test']['difference'], dtype=torch.float32)

train_image = torch.tensor(stamps_dataset['Train']['images'], dtype=torch.float32)
validation_image = torch.tensor(stamps_dataset['Validation']['images'], dtype=torch.float32)
test_image = torch.tensor(stamps_dataset['Test']['images'], dtype=torch.float32)

train_class_0 = torch.tensor(stamps_dataset['Train']['class'], dtype=torch.float32)
validation_class_0 = torch.tensor(stamps_dataset['Validation']['class'], dtype=torch.float32)
test_class_0 = torch.tensor(stamps_dataset['Test']['class'], dtype=torch.float32)

train_template = train_template.unsqueeze(1).repeat(1, 5, 1, 1)
validation_template = validation_template.unsqueeze(1).repeat(1, 5, 1, 1)
test_template = test_template.unsqueeze(1).repeat(1, 5, 1, 1)


train_dataset = torch.stack((train_template, train_image, train_difference), dim=3  )
validation_dataset = torch.stack((validation_template, validation_difference, validation_difference), dim=3)
test_dataset = torch.stack((test_template, test_image, test_difference), dim=3)

train_template = train_template.unsqueeze(2)  # (samples, 5, 1, 21, 21)
train_image = train_image.unsqueeze(2)        
train_difference = train_difference.unsqueeze(2)  

validation_template = validation_template.unsqueeze(2)
validation_image = validation_image.unsqueeze(2)
validation_difference = validation_difference.unsqueeze(2)

test_template = test_template.unsqueeze(2)
test_image = test_image.unsqueeze(2)
test_difference = test_difference.unsqueeze(2)

# Apilar los tensores a lo largo de la dimensión correcta
train_dataset = torch.cat((train_template, train_image, train_difference), dim=2)
validation_dataset = torch.cat((validation_template, validation_image, validation_difference), dim=2)
test_dataset = torch.cat((test_template, test_image, test_difference), dim=2)

# Crear los conjuntos de datos
train_dataset = TensorDataset(train_dataset, train_class_0)
validation_dataset = TensorDataset(validation_dataset, validation_class_0)
test_dataset = TensorDataset(test_dataset, test_class_0)

# Verificar las dimensiones de los conjuntos de datos
print(train_dataset.tensors[0].shape)  # (samples, 5, 3, 21, 21)
print(validation_dataset.tensors[0].shape)
print(test_dataset.tensors[0].shape)



torch.Size([123227, 5, 3, 21, 21])
torch.Size([600, 5, 3, 21, 21])
torch.Size([600, 5, 3, 21, 21])


In [94]:
importlib.reload(ae_recurrente)
importlib.reload(train_recurrent)
importlib.reload(metrics)



model = ae_recurrente.AE_recurrent(latent_dim=21,
                                    n_channels=3,
                                    sequence_length=5,
                                    hidden_dim=64,
                                    num_layers=1,
                                    name='AE_Recurrente_test')


#static parameters
max_epochs = 50
criterion = ae.loss_function
batch_size = 150 #100
lr = 1.665e-4

augmentation = False
suffle = False
early_stopping = 15
use_gpu = False

curves_2_0, tiempo_2_0, mse_2_0 = train_recurrent.train_model(model,
                                            train_dataset,
                                            validation_dataset,
                                            test_dataset,
                                            max_epochs=max_epochs,
                                            criterion=criterion,
                                            batch_size=batch_size,
                                            lr=lr,
                                            augmentation=augmentation,
                                            shuffle_augmentation=suffle,
                                            early_stop=early_stopping,
                                            use_gpu=use_gpu)

                                    

Starting training ...
torch.Size([150, 5, 3, 21, 21])
x torch.Size([750, 3, 21, 21])
z torch.Size([150, 5, 21])


/home/juanpablo/universidad/proyecto/ZTF_alert_8a/src/model/ae.py:86: UserWarning: Using a target size (torch.Size([150, 5, 3, 21, 21])) that is different to the input size (torch.Size([150, 3, 21, 21])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  recon_loss = F.mse_loss(recon_x, x, reduction='mean')


RuntimeError: The size of tensor a (150) must match the size of tensor b (5) at non-singleton dimension 1